<a href="https://colab.research.google.com/github/wjleece/ai-reflector/blob/main/wjleece_Langchain_Reflector_2_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -U --quiet  langchain langgraph
%pip install -U --quiet tavily-python
%pip install -U --quiet fireworks-ai
%pip install -U --quiet langchain_fireworks

In [ ]:
import os
import getpass
import re
from langchain_fireworks import Fireworks
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import LLMChain

In [ ]:
def set_env_var(var: str) -> None:
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Enter {var}: ")

# Set up environment variables
set_env_var("LANGCHAIN_API_KEY")
set_env_var("FIREWORKS_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Reflection"

In [ ]:
# Initialize ChatFireworks
base_llm = Fireworks(
    model="accounts/fireworks/models/llama-v3p1-70b-instruct",
    temperature=0.7,
    max_tokens=16384
)

reflector_llm = Fireworks(
    model="accounts/fireworks/models/mixtral-8x7b-instruct",
    temperature=0.7,
    max_tokens=32768
)


In [ ]:
# Define prompts
essay_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an essay assistant tasked with writing excellent 5-paragraph essays. "
               "Generate the best essay possible for the user's request. "
               "If you receive feedback or a critique of the essay, incorporate the feedback and rewrite the essay. "
               "If you rewrite the essay, preface it with 'Revised Essay:' to make it clear that what follows is a revision. "
               "If you have no feedback or critiques, you do not need to rewrite the essay."),
    MessagesPlaceholder(variable_name="messages")
])

reflection_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a teacher grading an essay submission. Generate critique and recommendations for the user's submission. "
               "Provide detailed recommendations, including requests for length, depth, style, etc. "
               "Make sure to include an overall score from 0 - 100 prefaced by the indicator 'Score:' for the essay prominently as either the first line or title of the critique."
               "Make sure the score is written as a number and not as a fraction."),
    MessagesPlaceholder(variable_name="messages")
])

In [ ]:
# Create LLMChains
generate_chain = essay_prompt | base_llm

reflect_chain = reflection_prompt | reflector_llm

In [ ]:
def extract_score(reflection):
    print("Debug: Reflection content:", reflection)  # Debug statement

    # Look for patterns like "Score: X" or "Score: X/Y" or "X/100" or just a number as the LLM returns all types of values, even when explicitly told not to return certain formats
    score_patterns = [
        r"Score:\s*(\d+)(?:/\d+)?",
        r"(\d+)/100",
        r"^(\d+)$"
    ]

    for pattern in score_patterns:
        match = re.search(pattern, reflection, re.MULTILINE)
        if match:
            try:
                score = int(match.group(1))
                print("Debug: Extracted score:", score)  # Debug statement
                return score
            except ValueError:
                print("Debug: Failed to convert matched score to integer")

    print("Debug: No valid score pattern found")
    print("Error extracting score. Using None.")
    return None

In [ ]:
essay = ""

def generate_and_reflect_essay():

    request = "Write an essay on the history of Canada." #time to learn about your northern neighbour

   #write initial AI-generated essay
    essay = generate_chain.invoke({"messages": [HumanMessage(content=request)]})


    #reflect on initial AI-generated essay
    reflection = reflect_chain.invoke({
        "messages": [
            HumanMessage(content=request),
            AIMessage(content=essay)
        ]
    })

    #get the essay score
    score = extract_score(reflection)

    return request, essay, reflection, score

In [ ]:
request, essay, reflection, score = generate_and_reflect_essay()

In [ ]:
request

In [ ]:
essay

In [ ]:
reflection

In [ ]:
score

In [ ]:
def iterate(request, essay, reflection, score):
    revised_essay = essay
    target_score = 95
    revision_count = 0
    max_revisions = 10
    min_essay_length = 50

    while (score is None or score < target_score) and revision_count < max_revisions:
        revised_essay = generate_chain.invoke({"messages": [
            HumanMessage(content=request),
            AIMessage(content=revised_essay),
            HumanMessage(content=reflection)
        ]})

        new_essay = revised_essay

        words = new_essay.strip().split()
        essay_length = len(words)

        if essay_length >= min_essay_length:
            revised_essay = new_essay
            revision_count += 1
            print(f"Revision count: {revision_count}")
            print(f"Essay length: {essay_length}")

            reflection_response = reflect_chain.invoke({"messages": [
                HumanMessage(content=request),
                AIMessage(content=revised_essay)
            ]})
            reflection = reflection_response

            score = extract_score(reflection)
            print(f"Essay score: {score}\n")
        else:
            print(f"Generated essay too short ({essay_length} words). Retrying...")

    if revision_count == max_revisions:
        print("Maximum number of revisions reached.")

    return request, revised_essay, reflection, score

In [ ]:
def main():
    request, essay, reflection, score = generate_and_reflect_essay()
    request, revised_essay, reflection, score = iterate(request, essay, reflection, score)
    print(f"Final Essay:\n{revised_essay}")

if __name__ == "__main__":
    main()